In [2]:
import pandas as pd
df = pd.read_pickle('dataset_with_hadm.pkl')
df.head()

,SUBJECT_ID,HADM_ID,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,22,165315,2,0,65,462.0,0.0,24.0,22.0,1.0,...,0,0,0,0,0,0,0,1,1,0
1,23,152223,6,1,71,462.0,0.0,24.0,26.0,1.0,...,0,0,0,0,0,0,0,0,0,1
2,23,124321,7,1,75,462.0,1.0,24.0,27.0,1.0,...,0,0,0,0,0,1,0,0,0,1
3,24,161859,3,1,39,462.0,0.0,24.0,27.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,25,129635,4,1,59,462.0,0.0,24.0,26.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_nohadm = df.drop(['HADM_ID','SUBJECT_ID'], axis=1)

In [4]:
x = df_nohadm.iloc[:,:483]
y = df_nohadm.iloc[:,483:]
print(x.columns,y.columns)

Index(['ADMISSION_DAYS',         'GENDER',            'AGE',            50801,
                  50802,            50803,            50804,            50805,
                  50806,            50808,
       ...
                  51512,            51513,            51514,            51515,
                  51516,            51517,            51518,            51519,
                  51529,            51532],
      dtype='object', length=483) Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')


In [36]:
seed = 15
from sklearn.model_selection import train_test_split
x_tra,x_test,y_tra,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)
x_train,x_val,y_train,y_val = train_test_split(x_tra,y_tra,test_size=0.25,random_state=seed) 
# i wanna have train,val,test ya cool

In [37]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss


In [52]:
n_folds = 5

def aucroc(y_val,y_pred):
    #kf = KFold(n_folds, shuffle=True, random_state=seed).get_n_splits(x_train.values)
    auc = roc_auc_score(y_val, y_pred) # oreder is ok? its y_true,y_pred okk
    return(auc)
def ham(y_val,y_pred):
    ham = hamming_loss(y_val,y_pred)
    return ham
def jack(y_val,y_pred):
    macro = jaccard_score(y_val,y_pred,'macro')
    micro = jaccard_score(y_val,y_pred,'micro')
    return macro
# jaccard also put

In [39]:
class average_models(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)*y.shape[1]))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred    # we need to mostly change here only
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [40]:
class average_models2(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)  #

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB 
gnb = OneVsRestClassifier(GaussianNB()) 
xgb_model = OneVsRestClassifier(XGBClassifier(1000,learning_rate=0.15,n_jobs=-1)) #make it lesser..
lgbm_model = OneVsRestClassifier(LGBMClassifier(1000,learning_rate=0.15,n_jobs=-1))
cat_model = OneVsRestClassifier(CatBoostClassifier(1000,learning_rate=0.15))
rf = OneVsRestClassifier(RandomForestClassifier(2000,n_jobs=-1))
# eh? simply i tried lol some thought just flashed that i should maybe try this haan but u didnt fit lol
#the others also arent fitting oops

In [43]:
import numpy as np
avg_models = average_models2(models = (xgb_model, rf, cat_model,gnb))           #                                      meta_model = rf)
#
#score = aucroc(avg_models)
#print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

avg_models.fit(x_train, y_train)
stacked_train_pred = avg_models.predict(x_train).reshape(-1,1)
stacked_pred = avg_models.predict(x_val).reshape(-1,1)
print(aucroc(y_train,stacked_train_pred))
print(aucroc(y_val,stacked_pred))

0:	learn: 0.5127492	total: 18.5ms	remaining: 18.4s
1:	learn: 0.3931128	total: 37ms	remaining: 18.4s
2:	learn: 0.3219603	total: 63.9ms	remaining: 21.2s
3:	learn: 0.2747478	total: 89.7ms	remaining: 22.3s
4:	learn: 0.2421497	total: 110ms	remaining: 21.9s
5:	learn: 0.2181668	total: 130ms	remaining: 21.5s
6:	learn: 0.2047383	total: 145ms	remaining: 20.6s
7:	learn: 0.1910030	total: 168ms	remaining: 20.8s
8:	learn: 0.1829005	total: 197ms	remaining: 21.7s
9:	learn: 0.1720993	total: 216ms	remaining: 21.4s
10:	learn: 0.1674329	total: 238ms	remaining: 21.4s
11:	learn: 0.1583568	total: 256ms	remaining: 21.1s
12:	learn: 0.1529765	total: 273ms	remaining: 20.8s
13:	learn: 0.1480326	total: 293ms	remaining: 20.7s
14:	learn: 0.1436350	total: 313ms	remaining: 20.6s
15:	learn: 0.1393142	total: 334ms	remaining: 20.5s
16:	learn: 0.1355496	total: 353ms	remaining: 20.4s
17:	learn: 0.1317537	total: 374ms	remaining: 20.4s
18:	learn: 0.1309338	total: 397ms	remaining: 20.5s
19:	learn: 0.1273914	total: 415ms	remai

176:	learn: 0.0157364	total: 2.69s	remaining: 12.5s
177:	learn: 0.0155931	total: 2.7s	remaining: 12.5s
178:	learn: 0.0155235	total: 2.71s	remaining: 12.4s
179:	learn: 0.0154520	total: 2.72s	remaining: 12.4s
180:	learn: 0.0153784	total: 2.74s	remaining: 12.4s
181:	learn: 0.0151728	total: 2.75s	remaining: 12.4s
182:	learn: 0.0150303	total: 2.76s	remaining: 12.3s
183:	learn: 0.0147529	total: 2.77s	remaining: 12.3s
184:	learn: 0.0146044	total: 2.79s	remaining: 12.3s
185:	learn: 0.0144043	total: 2.8s	remaining: 12.3s
186:	learn: 0.0143679	total: 2.82s	remaining: 12.3s
187:	learn: 0.0142407	total: 2.83s	remaining: 12.2s
188:	learn: 0.0142108	total: 2.84s	remaining: 12.2s
189:	learn: 0.0141688	total: 2.86s	remaining: 12.2s
190:	learn: 0.0139605	total: 2.87s	remaining: 12.2s
191:	learn: 0.0138554	total: 2.88s	remaining: 12.1s
192:	learn: 0.0135031	total: 2.9s	remaining: 12.1s
193:	learn: 0.0134236	total: 2.91s	remaining: 12.1s
194:	learn: 0.0132160	total: 2.92s	remaining: 12.1s
195:	learn: 0.0

345:	learn: 0.0053588	total: 4.93s	remaining: 9.33s
346:	learn: 0.0053125	total: 4.95s	remaining: 9.31s
347:	learn: 0.0052653	total: 4.96s	remaining: 9.29s
348:	learn: 0.0052421	total: 4.97s	remaining: 9.28s
349:	learn: 0.0052367	total: 4.98s	remaining: 9.26s
350:	learn: 0.0052294	total: 5s	remaining: 9.24s
351:	learn: 0.0052004	total: 5.01s	remaining: 9.23s
352:	learn: 0.0051605	total: 5.05s	remaining: 9.25s
353:	learn: 0.0051523	total: 5.07s	remaining: 9.26s
354:	learn: 0.0051292	total: 5.09s	remaining: 9.25s
355:	learn: 0.0050481	total: 5.11s	remaining: 9.24s
356:	learn: 0.0050332	total: 5.13s	remaining: 9.23s
357:	learn: 0.0050094	total: 5.15s	remaining: 9.24s
358:	learn: 0.0049882	total: 5.17s	remaining: 9.24s
359:	learn: 0.0049723	total: 5.2s	remaining: 9.24s
360:	learn: 0.0049372	total: 5.22s	remaining: 9.24s
361:	learn: 0.0049154	total: 5.23s	remaining: 9.22s
362:	learn: 0.0048681	total: 5.25s	remaining: 9.22s
363:	learn: 0.0048617	total: 5.27s	remaining: 9.2s
364:	learn: 0.004

509:	learn: 0.0028633	total: 7.17s	remaining: 6.89s
510:	learn: 0.0028540	total: 7.19s	remaining: 6.88s
511:	learn: 0.0028517	total: 7.2s	remaining: 6.86s
512:	learn: 0.0028468	total: 7.21s	remaining: 6.84s
513:	learn: 0.0028419	total: 7.22s	remaining: 6.83s
514:	learn: 0.0028373	total: 7.24s	remaining: 6.81s
515:	learn: 0.0028346	total: 7.25s	remaining: 6.8s
516:	learn: 0.0028197	total: 7.27s	remaining: 6.79s
517:	learn: 0.0028178	total: 7.28s	remaining: 6.77s
518:	learn: 0.0028030	total: 7.29s	remaining: 6.76s
519:	learn: 0.0027639	total: 7.31s	remaining: 6.75s
520:	learn: 0.0027444	total: 7.32s	remaining: 6.73s
521:	learn: 0.0027421	total: 7.34s	remaining: 6.72s
522:	learn: 0.0027344	total: 7.36s	remaining: 6.71s
523:	learn: 0.0027319	total: 7.37s	remaining: 6.7s
524:	learn: 0.0027252	total: 7.39s	remaining: 6.68s
525:	learn: 0.0027179	total: 7.4s	remaining: 6.67s
526:	learn: 0.0027111	total: 7.41s	remaining: 6.65s
527:	learn: 0.0026955	total: 7.43s	remaining: 6.64s
528:	learn: 0.00

675:	learn: 0.0018490	total: 9.61s	remaining: 4.6s
676:	learn: 0.0018478	total: 9.62s	remaining: 4.59s
677:	learn: 0.0018397	total: 9.64s	remaining: 4.58s
678:	learn: 0.0018385	total: 9.66s	remaining: 4.57s
679:	learn: 0.0018343	total: 9.68s	remaining: 4.55s
680:	learn: 0.0018267	total: 9.7s	remaining: 4.54s
681:	learn: 0.0018247	total: 9.71s	remaining: 4.53s
682:	learn: 0.0018223	total: 9.73s	remaining: 4.51s
683:	learn: 0.0018113	total: 9.74s	remaining: 4.5s
684:	learn: 0.0018104	total: 9.76s	remaining: 4.49s
685:	learn: 0.0018097	total: 9.81s	remaining: 4.49s
686:	learn: 0.0018090	total: 9.83s	remaining: 4.48s
687:	learn: 0.0018084	total: 9.86s	remaining: 4.47s
688:	learn: 0.0018077	total: 9.88s	remaining: 4.46s
689:	learn: 0.0018070	total: 9.9s	remaining: 4.45s
690:	learn: 0.0018041	total: 9.92s	remaining: 4.44s
691:	learn: 0.0018017	total: 9.95s	remaining: 4.43s
692:	learn: 0.0018001	total: 9.96s	remaining: 4.41s
693:	learn: 0.0017994	total: 9.98s	remaining: 4.4s
694:	learn: 0.001

...

235:	learn: 0.0609871	total: 4.17s	remaining: 13.5s
236:	learn: 0.0609191	total: 4.18s	remaining: 13.5s
237:	learn: 0.0607116	total: 4.19s	remaining: 13.4s
238:	learn: 0.0603183	total: 4.21s	remaining: 13.4s
239:	learn: 0.0595904	total: 4.22s	remaining: 13.4s
240:	learn: 0.0589522	total: 4.23s	remaining: 13.3s
241:	learn: 0.0581122	total: 4.24s	remaining: 13.3s
242:	learn: 0.0579340	total: 4.25s	remaining: 13.3s
243:	learn: 0.0574430	total: 4.27s	remaining: 13.2s
244:	learn: 0.0571051	total: 4.28s	remaining: 13.2s
245:	learn: 0.0565975	total: 4.29s	remaining: 13.1s
246:	learn: 0.0560250	total: 4.3s	remaining: 13.1s
247:	learn: 0.0557000	total: 4.32s	remaining: 13.1s
248:	learn: 0.0552663	total: 4.33s	remaining: 13.1s
249:	learn: 0.0552374	total: 4.34s	remaining: 13s
250:	learn: 0.0548902	total: 4.36s	remaining: 13s
251:	learn: 0.0547096	total: 4.36s	remaining: 13s
252:	learn: 0.0544323	total: 4.38s	remaining: 12.9s
253:	learn: 0.0543708	total: 4.38s	remaining: 12.9s
254:	learn: 0.05361

394:	learn: 0.0237931	total: 6.42s	remaining: 9.83s
395:	learn: 0.0237154	total: 6.43s	remaining: 9.82s
396:	learn: 0.0236022	total: 6.45s	remaining: 9.79s
397:	learn: 0.0233694	total: 6.46s	remaining: 9.78s
398:	learn: 0.0232965	total: 6.48s	remaining: 9.76s
399:	learn: 0.0231351	total: 6.49s	remaining: 9.73s
400:	learn: 0.0231134	total: 6.5s	remaining: 9.71s
401:	learn: 0.0230550	total: 6.51s	remaining: 9.69s
402:	learn: 0.0228390	total: 6.53s	remaining: 9.67s
403:	learn: 0.0228322	total: 6.54s	remaining: 9.65s
404:	learn: 0.0228263	total: 6.56s	remaining: 9.63s
405:	learn: 0.0228157	total: 6.57s	remaining: 9.62s
406:	learn: 0.0226493	total: 6.59s	remaining: 9.6s
407:	learn: 0.0225706	total: 6.6s	remaining: 9.58s
408:	learn: 0.0224706	total: 6.61s	remaining: 9.55s
409:	learn: 0.0224179	total: 6.62s	remaining: 9.53s
410:	learn: 0.0223137	total: 6.63s	remaining: 9.51s
411:	learn: 0.0221568	total: 6.64s	remaining: 9.48s
412:	learn: 0.0221314	total: 6.66s	remaining: 9.47s
413:	learn: 0.0

560:	learn: 0.0145863	total: 8.84s	remaining: 6.92s
561:	learn: 0.0145826	total: 8.86s	remaining: 6.9s
562:	learn: 0.0145117	total: 8.87s	remaining: 6.88s
563:	learn: 0.0145095	total: 8.88s	remaining: 6.86s
564:	learn: 0.0145067	total: 8.89s	remaining: 6.85s
565:	learn: 0.0144878	total: 8.91s	remaining: 6.83s
566:	learn: 0.0144626	total: 8.93s	remaining: 6.82s
567:	learn: 0.0144613	total: 8.94s	remaining: 6.8s
568:	learn: 0.0144466	total: 8.96s	remaining: 6.78s
569:	learn: 0.0144452	total: 8.97s	remaining: 6.76s
570:	learn: 0.0144440	total: 8.98s	remaining: 6.75s
571:	learn: 0.0144427	total: 9s	remaining: 6.73s
572:	learn: 0.0144317	total: 9.02s	remaining: 6.72s
573:	learn: 0.0144295	total: 9.03s	remaining: 6.7s
574:	learn: 0.0144274	total: 9.04s	remaining: 6.68s
575:	learn: 0.0144056	total: 9.05s	remaining: 6.67s
576:	learn: 0.0143998	total: 9.07s	remaining: 6.65s
577:	learn: 0.0143689	total: 9.08s	remaining: 6.63s
578:	learn: 0.0143674	total: 9.09s	remaining: 6.61s
579:	learn: 0.0143

720:	learn: 0.0112735	total: 11.1s	remaining: 4.3s
721:	learn: 0.0112727	total: 11.1s	remaining: 4.28s
722:	learn: 0.0112721	total: 11.1s	remaining: 4.26s
723:	learn: 0.0111443	total: 11.1s	remaining: 4.25s
724:	learn: 0.0111438	total: 11.2s	remaining: 4.23s
725:	learn: 0.0111430	total: 11.2s	remaining: 4.22s
726:	learn: 0.0111405	total: 11.2s	remaining: 4.2s
727:	learn: 0.0111378	total: 11.2s	remaining: 4.18s
728:	learn: 0.0111123	total: 11.2s	remaining: 4.17s
729:	learn: 0.0110716	total: 11.2s	remaining: 4.15s
730:	learn: 0.0110700	total: 11.2s	remaining: 4.14s
731:	learn: 0.0110694	total: 11.3s	remaining: 4.12s
732:	learn: 0.0110688	total: 11.3s	remaining: 4.11s
733:	learn: 0.0110682	total: 11.3s	remaining: 4.09s
734:	learn: 0.0110677	total: 11.3s	remaining: 4.07s
735:	learn: 0.0110672	total: 11.3s	remaining: 4.06s
736:	learn: 0.0110647	total: 11.3s	remaining: 4.04s
737:	learn: 0.0110496	total: 11.3s	remaining: 4.02s
738:	learn: 0.0110488	total: 11.3s	remaining: 4s
739:	learn: 0.011

888:	learn: 0.0094243	total: 13.8s	remaining: 1.72s
889:	learn: 0.0094235	total: 13.8s	remaining: 1.7s
890:	learn: 0.0094220	total: 13.8s	remaining: 1.69s
891:	learn: 0.0094217	total: 13.8s	remaining: 1.67s
892:	learn: 0.0094214	total: 13.8s	remaining: 1.65s
893:	learn: 0.0094210	total: 13.8s	remaining: 1.64s
894:	learn: 0.0093808	total: 13.8s	remaining: 1.62s
895:	learn: 0.0093796	total: 13.9s	remaining: 1.61s
896:	learn: 0.0093783	total: 13.9s	remaining: 1.59s
897:	learn: 0.0093769	total: 13.9s	remaining: 1.58s
898:	learn: 0.0093701	total: 13.9s	remaining: 1.56s
899:	learn: 0.0093688	total: 13.9s	remaining: 1.54s
900:	learn: 0.0093675	total: 13.9s	remaining: 1.53s
901:	learn: 0.0093660	total: 13.9s	remaining: 1.51s
902:	learn: 0.0093649	total: 14s	remaining: 1.5s
903:	learn: 0.0093466	total: 14s	remaining: 1.48s
904:	learn: 0.0093190	total: 14s	remaining: 1.47s
905:	learn: 0.0092874	total: 14s	remaining: 1.45s
906:	learn: 0.0092407	total: 14s	remaining: 1.44s
907:	learn: 0.0092250	to

In [53]:
pred = avg_models.predict(x_test).reshape(-1,1) 
print(aucroc(y_test,pred))
print(jack(y_test,pred))
print(ham(y_test,pred))#this may throw erro

0.8313240490542506


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous targets

In [54]:
stacked_train_pred.shape

(1275, 1)

In [55]:
y_train.shape

(1275, 20)

In [ ]:
averaged_models = average_models2(models = (xgb_model, lgbm_model, cat_model))

score = aucroc(averaged_models)#its the same as the one we editted below ok
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))